In [1]:
import parse_tfl_json
import google

In [2]:
import pandas as pd 
from mylibrary.connections import cursor, conn, engine, Automapped_Base, session

sql = """
select * from tt_h.stations_journeytime 
where tfl_message = 'ok' 
limit 100
"""
df = pd.read_sql(sql, conn)

/Users/robinlinacre/anaconda/lib/python3.6/site-packages/sqlalchemy/dialects/postgresql/base.py:2526: SAWarning: Did not recognize type 'geometry' of column 'geom'
  (attype, name))


In [3]:
from process_journeys import get_london_icscodes, add_cycle_and_total_time, remove_journeys_not_arriving_clondon
from google import get_cycle_info
import parse_tfl_json
import json

In [4]:
# df = df.loc[8:9,:]

In [5]:
for r in df.iterrows():
    index = r[0]
    row = r[1]

    try:
        j1 = row["tfl_response"]
        j2 = json.loads(j1)
    except:
        print("parse problem")
        continue
    
    try:
        journeys = j2["journeys"]
    except:
        print("no journeys in json")
        continue
    
    
    for journey in journeys:
        legs = journey["legs"]
        journey["legs"] = parse_tfl_json.remove_non_national_rail(legs)
    

    
    # Keep only journeys with a national rail component
    journeys = [j for j in journeys if len(j["legs"]) > 0]
    
    
    
    # Keep only journeys that end in central London
    
    journeys = remove_journeys_not_arriving_clondon(journeys)

    
    if len(journeys) == 0:
        continue

    journeys = add_cycle_and_total_time(journeys, row["tfl_dest"])

    journeys = sorted(journeys, key = lambda x: x["total_time"])

    journey = journeys[0]
    legs = journey["legs"]

    summary = parse_tfl_json.summarise_legs(legs)
    df.loc[index, "natrail_journey_summary"] = summary

    df.loc[index, "natrail_train_changes"] = parse_tfl_json.num_changes(legs)
    df.loc[index, "final_arrival"] = parse_tfl_json.final_arrival(legs)

    travel_time_minutes = parse_tfl_json.get_total_travel_time(legs) 
    df.loc[index, "natrail_journey_minutes"] = travel_time_minutes

    lat_lng = parse_tfl_json.lat_lng_from_legs(legs)

    df.loc[index, "depart_lat"] = lat_lng["depart"]["lat"]
    df.loc[index, "depart_lng"] = lat_lng["depart"]["lng"]
    df.loc[index, "arrive_lat"] = lat_lng["arrive"]["lat"]
    df.loc[index, "arrive_lng"] = lat_lng["arrive"]["lng"]

    df.loc[index, "cycle_minutes"] = journey["cycle_minutes"]
    df.loc[index, "cycle_miles"] = journey["cycle_miles"]
    df.loc[index, "total_journeytime"] = journey["cycle_minutes"] + travel_time_minutes

    
    
cols = [c for c in df.columns if c not in ["id", "tfl_request", "tfl_response", "querydict"]]
df2 = df[cols]

In [9]:
df3 = df2.sort_values(["station_name", "total_journeytime"]).drop_duplicates("station_name")
df3

,nlc,icscode,station_name,tfl_message,tfl_dest,natrail_journey_summary,natrail_train_changes,final_arrival,natrail_journey_minutes,depart_lat,depart_lng,arrive_lat,arrive_lng,cycle_minutes,cycle_miles,total_journeytime
33,9020,90001023,Barry Links,ok,SW1H9AJ,Barry Links Rail Station → Dundee Rail Station...,2.0,London King's Cross Rail Station,402.0,56.493147,-2.745442,51.531579,-0.123243,19.895412,4.642263,421.895412
81,1006,90000594,Birmingham Snow Hill,ok,SW1H9AJ,Birmingham Snow Hill Rail Station → Birmingham...,1.0,Euston Station,92.0,52.482373,-1.898502,51.528952,-0.135000,17.235056,4.021513,109.235056
49,5626,1001511,Bookham,ok,SW1H9AJ,Bookham Rail Station → Vauxhall Rail Station,0.0,Vauxhall Rail Station,44.0,51.288737,-0.384035,51.485911,-0.122411,9.083556,2.119496,53.083556
5,5006,90001027,Broadstairs,ok,SW1H9AJ,Broadstairs Rail Station → St Pancras Internat...,0.0,St Pancras International Station,84.0,51.360673,1.433561,51.532313,-0.126400,18.859498,4.400549,102.859498
40,3502,90001146,Bugle,ok,SW1H9AJ,Bugle Rail Station → Par Rail Station → Paddin...,1.0,Paddington Station,289.0,50.399949,-4.791795,51.516993,-0.177388,11.964942,2.791820,300.964942
14,8621,90001294,Carrbridge,ok,SW1H9AJ,Carrbridge Rail Station → Perth Rail Station →...,2.0,London King's Cross Rail Station,455.0,57.279500,-3.828203,51.531579,-0.123243,19.895412,4.642263,474.895412
20,2762,90001379,Chapel-En-Le-Frith,ok,SW1H9AJ,Chapel-en-le-Frith Rail Station → Stockport Ra...,1.0,Euston Station,160.0,53.312229,-1.918762,51.528952,-0.135000,17.235056,4.021513,177.235056
1,4876,90001586,Colwall,ok,SW1H9AJ,Colwall Rail Station → Birmingham New Street R...,1.0,Euston Station,170.0,52.079867,-2.356950,51.528952,-0.135000,17.235056,4.021513,187.235056
46,7349,90002536,Elmswell,ok,E145HP,Elmswell Rail Station → Ipswich Rail Station →...,1.0,Liverpool Street Station,103.0,52.238039,0.912589,51.518380,-0.081093,15.474801,3.610787,118.474801
6,3776,90002610,Eskbank,ok,SW1H9AJ,Eskbank Rail Station → Edinburgh Rail Station ...,1.0,London King's Cross Rail Station,285.0,55.881801,-3.083073,51.531579,-0.123243,19.895412,4.642263,304.895412


In [ ]:
# Want to update all the names to be cw specific 
df5 = df4.rename(columns={"depart_lat" : "lat", "depart_lng": "lng", "station_name": "name"})

In [ ]:
df5.to_csv("cw_to_map.csv", encoding="utf-8", index=False)

In [ ]:
# Now want to spread so that we have one row per station, then metrics for both cw and pw on that row
